In [ ]:
# !pip3 install transformers

In [ ]:
# ! nvidia-smi

In [ ]:
# !pip3 install tqdm pandas

In [ ]:
# ! pip install gdown

In [130]:

import torch
import pandas as pd
from tqdm import tqdm
import numpy as np


In [2]:
from transformers import BitsAndBytesConfig
from transformers import AutoModelForCausalLM, AutoTokenizer



# Get the type
compute_dtype = getattr(torch, "float16")

# BitsAndBytesConfig int-4 config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype
)

hf_model_repo = "meta-llama/Llama-2-7b-chat-hf"

# Get the tokenizer
tokenizer = AutoTokenizer.from_pretrained(hf_model_repo)

# Load the model
model = AutoModelForCausalLM.from_pretrained(hf_model_repo,
                                             quantization_config=bnb_config,
                                             device_map="auto")

Loading checkpoint shards: 100%|██████████| 2/2 [00:05<00:00,  2.87s/it]
You are calling `save_pretrained` to a 4-bit converted model, but your `bitsandbytes` version doesn't support it. If you want to save 4-bit models, make sure to have `bitsandbytes>=0.41.3` installed.


In [3]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): Lla

In [4]:
! ls

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
QnA_Generator-coaxnn-paper.ipynb	finetuning_qna_paper
all_coaxnn_paper_responsez_1st_run.txt	pretraining_paper
coaxnn_paper_qna_Ref_train.csv		train_coaxnn_paper.csv


In [5]:
all_csvs = [
    "data/train_coaxnn_paper.csv",
]

In [6]:
# all_responsez_file = open("all_coaxnn_paper_responsez_1st_run.txt", 'w')
# all_responsez = {"text": [], "response": []}

In [7]:
csvf = "data/train_coaxnn_paper.csv"
text_chunks = pd.read_csv(csvf)

paper_text_chunks = text_chunks['text'][:94]
refs_text_chunks = text_chunks['text'][95:]

# paper_text_chunks = paper_text_chunks[-5:]

print("paper_text_chunks: ", len(paper_text_chunks))


paper_text_chunks:  94


In [12]:


# for csvf in all_csvs:


print("csvf = ", csvf)

ixcounter = -1

for txtch in tqdm(paper_text_chunks, f"processing_{csvf}"):

    txtch = txtch.replace('-\n','')

    # prompt = \
    # f"""
    # Instruction: Understand the below text and generate one question and answer pair.\n
    # Input:\n
    # {txtch}\n
    # Output:\n
    # """

    prompt = \
    f"""
    Instruction: Generate three question and answer pairs for the below text. All questions must start with "Question". All answers must start wtih "Answer"\n
    Input:\n
    {txtch}\n
    Output:\n
    """

    # print(prompt)

    # print("\n\n")

    # Generate response
    input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids
    outputs = model.generate(input_ids=input_ids,
                            max_new_tokens=200,
                            temperature=0.6)

    # gen_tokens; exclude input tokens from the final decoded output
    gen_tokens = outputs[:, input_ids.shape[1]:]
    response = tokenizer.batch_decode(gen_tokens, skip_special_tokens=True)[0]

    all_responsez["text"].append(txtch)
    all_responsez["response"].append(response)
    
    all_responsez_file.write(f"{response}\n++++++++++++++++++++++++++++++++++++++++++++++++++++\n")
    all_responsez_file.flush()

    # (running locally on RTX 2060) cool donw GPU before next run
    # time.sleep(6)


csvf =  train_coaxnn_paper.csv


processing_train_coaxnn_paper.csv:   0%|          | 0/94 [00:00<?, ?it/s]Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
/home/aribra/miniconda3/envs/llama2/lib/python3.12/site-packages/transformers/generation/utils.py:1659: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
processing_train_coaxnn_paper.csv: 100%|██████████| 94/94 [16:13<00:00, 10.35s/it]


In [99]:
if not os.path.exists():
    qna_df_responsez = pd.DataFrame(all_responsez)
    qna_df_responsez.to_csv("data/coaxnn_paper_qna_all_responsez.csv", index=False)

In [100]:
qna_df_responsez

,text,response
0,Contents lists available at ScienceDirect\n\nJ...,Question 1: What is the main contribution of t...
1,Efficient neural networks\nModel approximation...,Question 1: What are the approximate strategie...
2,"propose a novel model optimization framework, ...",Question: What is the main goal of the propose...
3,up to 1.53× speedup while reducing energy by u...,Question 1: What is the main goal of the text?...
4,to the limited resources.\n\nMany efforts have...,Question 1: What are the efforts made to enabl...
...,...,...
89,"ability to classify simple images. Besides, si...",Question 1: What is the main advantage of usin...
90,significantly reducing computation costs for n...,Question 1: What are the benefits of GA-based ...
91,"Therefore, the runtime overhead of the GA is n...",Question: What is the main advantage of CoAxNN...
92,"requirements of intelligent tasks. Moreover, t...",Question 1: What are the requirements of intel...


In [101]:
print(qna_df_responsez.iloc[30]['response'])

1. Question: What is the purpose of removing unimportant filters in the text?
Answer: The purpose of removing unimportant filters is to reduce computational costs.

    2. Question: What is the next step after evaluating the fitness of the corresponding individuals according to the accuracy and latency of the compressed multi-stage model?
Answer: The next step is to update the chromosome pool, generating the next generation of individuals.

    3. Question: What is the general approach of executing a neural network model?
Answer: The general approach of executing a neural network model is a one-staged approach, which processes all the inputs in the same manner, starting from the input operator and performing it operator by operator until the final exit operator.


In [36]:
import regex as re

In [102]:
def get_QA_pairs(qa_df):
    pairs = {"Question": [], "Answer": []}
    
    pattern = re.compile(r"Question\s*\d*:\s*(.*?)\s*Answer\s*\d*:\s*(.*?)(?=\n\s*\d*\.\s*Question|\n\s*Question|\Z)", re.DOTALL)

    for qa_response in qa_df['response']:

        matches = pattern.findall(qa_response)

        for question, answer in matches:
            # print(f"Question: {question.strip()}")
            if answer:
                # print(f"Answer: {answer.strip()}")
                pairs["Question"].append(question)
                pairs["Answer"].append(answer)
            else:
                print("skipped: ", question)
    return pairs

In [103]:
qa_pairs = get_QA_pairs(qna_df_responsez)

In [104]:
qa_pairs['Question']

['What is the main contribution of the paper?',
 'What are the keywords associated with this article?',
 'What is the main goal of the paper?',
 'What are the approximate strategies used in the study?',
 'What is the purpose of combining different approximate strategies?',
 'What is the proposed model optimization framework called?',
 'What is the main goal of the proposed model optimization framework?',
 'What are the principles of different approximate optimizations used in the proposed approach?',
 'What is the effectiveness of CoAxNN according to the experimental results?',
 'What is the main goal of the text?',
 'What are the limitations of current deep learning models?',
 'What are the potential benefits of on-device deep learning?',
 'What are the efforts made to enable efficient on-device deep learning?',
 'What are the challenges in combining different optimization strategies?',
 'What are the approximate strategies based on distinct principles?',
 'What is the main aim of thi

In [105]:
len(qa_pairs['Question']), len(qa_pairs['Answer'])

(257, 257)

In [106]:
qna_dfb = pd.DataFrame(qa_pairs)
# qna_dfb.to_csv("coaxnn_paper_qna_train.csv", index=False)

In [107]:
qna_dfb

,Question,Answer
0,What is the main contribution of the paper?,The paper proposes a new method for optimizing...
1,What are the keywords associated with this art...,The keywords associated with this article are ...
2,What is the main goal of the paper?,The main goal of the paper is to develop a new...
3,What are the approximate strategies used in th...,Neural network pruning.
4,What is the purpose of combining different app...,To improve the performance of on-device infere...
...,...,...
252,What are the limitations of CoAxNN?,The limitations of CoAxNN include the use of a...
253,What future studies will be explored in CoAxNN?,Future studies in CoAxNN will include explorin...
254,What are the different sensitives for model ac...,Different layers have different sensitives for...
255,What will be explored in future studies?,Setting different pruning ratios for different...


In [114]:
refs_qa_pairs = {"Question":[], "Answer":[]}

for ix, ref in enumerate(refs_text_chunks.values):
    q = f"What is reference [{ix+1}]?"
    refs_qa_pairs["Question"].append(q)
    refs_qa_pairs["Answer"].append(ref)

ref_qna_df = pd.DataFrame(refs_qa_pairs)
# ref_qna_df.to_csv("coaxnn_paper_qna_Ref_train.csv", index=False)

### paper text and references Combined

In [116]:
qna_train_coaxnn_train_DF = pd.concat([qna_dfb, ref_qna_df])
qna_train_coaxnn_train_DF

In [153]:
if not os.path.exist("data/coaxnn_paper_qna_train.csv"):
    qna_train_coaxnn_train_DF.to_csv("data/coaxnn_paper_qna_train.csv", index=False)

In [152]:
rand_ix = np.random.randint(len(qna_train_coaxnn_train_DF))
print(qna_train_coaxnn_train_DF.iloc[rand_ix].values)

['What is reference [30]?'
 'Reference [30]: Y. Qian, Z. He, Y. Wang, B. Wang, X. Ling, Z. Gu, H. Wang, S. Zeng, W. Swaileh,\nHierarchical threshold pruning based on uniform response criterion, IEEE Trans.\nNeural Netw. Learn. Syst. (2023).']


In [8]:
import pandas as pd

df = pd.read_csv("data/coaxnn_paper_qna_train.csv")

df['text_question'] = 'Question:\n' + df['Question']
df['text_answer'] = 'Answer:\n' + df['Answer']

In [11]:
print(df['text_answer'].iloc[0])

Answer:
The paper proposes a new method for optimizing on-device deep learning with conditional approximate neural networks.


In [13]:

from tqdm import tqdm

In [16]:
for input_question in tqdm(df['text_question']):
    print(input_question)

100%|███████████████████████████████████████████████| 308/308 [00:00<00:00, 236862.05it/s]

Question:
What is the main contribution of the paper?
Question:
What are the keywords associated with this article?
Question:
What is the main goal of the paper?
Question:
What are the approximate strategies used in the study?
Question:
What is the purpose of combining different approximate strategies?
Question:
What is the proposed model optimization framework called?
Question:
What is the main goal of the proposed model optimization framework?
Question:
What are the principles of different approximate optimizations used in the proposed approach?
Question:
What is the effectiveness of CoAxNN according to the experimental results?
Question:
What is the main goal of the text?
Question:
What are the limitations of current deep learning models?
Question:
What are the potential benefits of on-device deep learning?
Question:
What are the efforts made to enable efficient on-device deep learning?
Question:
What are the challenges in combining different optimization strategies?
Question:
What 

In [17]:
for input_question in tqdm(df['text_answer']):
    print(input_question)

100%|████████████████████████████████████████████████| 308/308 [00:00<00:00, 41727.63it/s]

Answer:
The paper proposes a new method for optimizing on-device deep learning with conditional approximate neural networks.
Answer:
The keywords associated with this article are on-device deep learning, efficient neural networks, model approximation and optimization.
Answer:
The main goal of the paper is to develop a new method for deploying deep neural networks on resource-constrained devices while reducing their computational complexity.
Answer:
Neural network pruning.
Answer:
To improve the performance of on-device inference.
Answer:
CoAxNN.
Answer:
The main goal of the proposed model optimization framework is to effectively combine different approximate strategies to facilitate on-device deep learning via model approximation.
Answer:
The principles of different approximate optimizations used in the proposed approach include the use of different approximation methods such as quantization, pruning, and knowledge distillation.
Answer:
The effectiveness of CoAxNN is demonstrated throu

In [27]:
df['text_question'].iloc[256]

'Question:\nWhat is the conclusion of the paper?'

In [29]:
list(df['text_question'].values[:256])

['Question:\nWhat is the main contribution of the paper?',
 'Question:\nWhat are the keywords associated with this article?',
 'Question:\nWhat is the main goal of the paper?',
 'Question:\nWhat are the approximate strategies used in the study?',
 'Question:\nWhat is the purpose of combining different approximate strategies?',
 'Question:\nWhat is the proposed model optimization framework called?',
 'Question:\nWhat is the main goal of the proposed model optimization framework?',
 'Question:\nWhat are the principles of different approximate optimizations used in the proposed approach?',
 'Question:\nWhat is the effectiveness of CoAxNN according to the experimental results?',
 'Question:\nWhat is the main goal of the text?',
 'Question:\nWhat are the limitations of current deep learning models?',
 'Question:\nWhat are the potential benefits of on-device deep learning?',
 'Question:\nWhat are the efforts made to enable efficient on-device deep learning?',
 'Question:\nWhat are the challe

In [31]:
list(df['text_answer'].values[:256])

['Answer:\nThe paper proposes a new method for optimizing on-device deep learning with conditional approximate neural networks.',
 'Answer:\nThe keywords associated with this article are on-device deep learning, efficient neural networks, model approximation and optimization.',
 'Answer:\nThe main goal of the paper is to develop a new method for deploying deep neural networks on resource-constrained devices while reducing their computational complexity.',
 'Answer:\nNeural network pruning.',
 'Answer:\nTo improve the performance of on-device inference.',
 'Answer:\nCoAxNN.',
 'Answer:\nThe main goal of the proposed model optimization framework is to effectively combine different approximate strategies to facilitate on-device deep learning via model approximation.',
 'Answer:\nThe principles of different approximate optimizations used in the proposed approach include the use of different approximation methods such as quantization, pruning, and knowledge distillation.',
 'Answer:\nThe ef

In [32]:
df[:256]

,Question,Answer,text_question,text_answer
0,What is the main contribution of the paper?,The paper proposes a new method for optimizing...,Question:\nWhat is the main contribution of th...,Answer:\nThe paper proposes a new method for o...
1,What are the keywords associated with this art...,The keywords associated with this article are ...,Question:\nWhat are the keywords associated wi...,Answer:\nThe keywords associated with this art...
2,What is the main goal of the paper?,The main goal of the paper is to develop a new...,Question:\nWhat is the main goal of the paper?,Answer:\nThe main goal of the paper is to deve...
3,What are the approximate strategies used in th...,Neural network pruning.,Question:\nWhat are the approximate strategies...,Answer:\nNeural network pruning.
4,What is the purpose of combining different app...,To improve the performance of on-device infere...,Question:\nWhat is the purpose of combining di...,Answer:\nTo improve the performance of on-devi...
...,...,...,...,...
251,What are the requirements of intelligent tasks?,The requirements of intelligent tasks include ...,Question:\nWhat are the requirements of intell...,Answer:\nThe requirements of intelligent tasks...
252,What are the limitations of CoAxNN?,The limitations of CoAxNN include the use of a...,Question:\nWhat are the limitations of CoAxNN?,Answer:\nThe limitations of CoAxNN include the...
253,What future studies will be explored in CoAxNN?,Future studies in CoAxNN will include explorin...,Question:\nWhat future studies will be explore...,Answer:\nFuture studies in CoAxNN will include...
254,What are the different sensitives for model ac...,Different layers have different sensitives for...,Question:\nWhat are the different sensitives f...,Answer:\nDifferent layers have different sensi...


In [36]:
# ! pip3 install evaluate
! pip install rouge_score

/usr/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py) ... done
Using cached six-1.16.0-py2.py3-none-any.whl (11 kB)
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24955 sha256=8629f48e85b36deedd1e3d3668a48765fe7de9d7de6dda940275caa4d3b5cb3e
  Stored in directory: /home/aribra/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip


In [37]:
import evaluate
eval_metric = evaluate.load("rouge")

In [45]:
references = list( df['text_answer'].values )
predictions = list( df['text_answer'].values )
m = eval_metric.compute(predictions = predictions, references = references)

In [46]:
mp[roug]

{'rouge1': 1.0, 'rouge2': 1.0, 'rougeL': 1.0, 'rougeLsum': 1.0}

In [51]:
for input_question, gt_answer in tqdm(zip(df['text_question'], df['text_answer'])):
    print(input_question, gt_answer)

308it [00:00, 44982.26it/s]

Question:
What is the main contribution of the paper? Answer:
The paper proposes a new method for optimizing on-device deep learning with conditional approximate neural networks.
Question:
What are the keywords associated with this article? Answer:
The keywords associated with this article are on-device deep learning, efficient neural networks, model approximation and optimization.
Question:
What is the main goal of the paper? Answer:
The main goal of the paper is to develop a new method for deploying deep neural networks on resource-constrained devices while reducing their computational complexity.
Question:
What are the approximate strategies used in the study? Answer:
Neural network pruning.
Question:
What is the purpose of combining different approximate strategies? Answer:
To improve the performance of on-device inference.
Question:
What is the proposed model optimization framework called? Answer:
CoAxNN.
Question:
What is the main goal of the proposed model optimization framework